In [1]:
# RUN ONCE ONLY!
import os
os.chdir("./src")

In [2]:
from comma_fixer.column import Column
from comma_fixer.fixer import Fixer
from comma_fixer.schema import Schema
import pandas as pd

# Creating a Schema

To create a Schema, we have to define all the columns with the column name and type, with additional arguments on whether the columns are nullable.

Each column will have a function to determine whether a given token can be placed within that column. 

For `datetime` columns, it is critical that the input is in the ISO 8601 datetime format, i.e. `%Y-%m-%d` or `YYYY-MM-DD`, as this is the format accepted 
by the `pandas` library used for storing the dataset before exporting to CSV.

For example, assume we have the entry string "1,Bob,Johnson,twenty three,False,", and are checking whether "twenty three" is suitable for the "age" column.
Since the "age" column only accepts numeric values, it will return False. However, if the entry string were "1,Bob,Johnson,23,False,", the column's verifier 
would return True for "23".

Other types of columns can be created as well, but a `pd.Series` object must be supplied to be able to create a `pd.DataFrame` when exporting to CSV. This requires 
importing `pandas`. All arguments must be given.



In [3]:
schema_1 = Schema.new(columns=[
    Column.numeric(name="id"),
    Column.string(name="firstname", is_nullable=False, has_commas=False, has_spaces=False),
    Column.string(name="lastname", is_nullable=False, has_commas=False, has_spaces=False),
    Column.numeric(name="age"),
    Column.new(name="cat_owner", data_type=bool, series_type=pd.Series(dtype=bool), is_nullable=False, has_commas=False, has_spaces=False, format=None),
    Column.string(name="cat_names", is_nullable=True, has_commas=True, has_spaces=True)
    ]
)

After creating a Schema, its contents can be displayed in a table format. However, newer columns can not be added into existing Schemas. 

In [4]:
schema_1.info()

,name,type,nullable,has commas,has spaces,format
0,id,int,False,False,False,None
1,firstname,str,False,False,False,None
2,lastname,str,False,False,False,None
3,age,int,False,False,False,None
4,cat_owner,bool,False,False,False,None
5,cat_names,str,True,True,True,None


# Fixer

After creating a Schema, it can be used to create a `Fixer`.

In [5]:
fixer_1 = Fixer.new(schema_1)

A file can be processed one at a time by passing in the filepath to the fixer. This will create a `Parsed` object where the processed, valid rows can be exported into a CSV file, and invalid rows can be viewed.

Primarily, invalid rows may occur if there are multiple ways of parsing the row to fit the schema, or there is no valid parsing. This may be a result of a weak, non-restrictive schema. To fix this, the schema should contain further restrictive elements such as RegEx formatting.

If enabled in `fix_file`, invalid rows can also print out their possible tokenisations for fixing.

## Example 1

The `example_1.csv` file only has one column with commas, so there should not be any invalid rows aside from rows which are missing values.

In [6]:
parsed_example_1 = fixer_1.fix_file(filepath="../examples/example_1.csv", skip_first_line=True, show_possible_parses=True)

No paths found at line index 8.
No paths found


File has been processed!
Number of total entries: 9            
 Number of invalid entries: 1


In [7]:
parsed_example_1.print_all_invalid_entries()

Index	Line entry
8	8,Pujan,,Sir,32,True,,,


The above line only creates an error since it is missing a value for the "lastname" column, which must not be null.

## Example 2

However, if there are multiple columns allowing commas consecutively, the fixer will be unable to parse rows as efficiently compared to other schemas, i.e. schemas where columns with commas are separated by a different type, such as numeric types.

An example is shown below.

In [8]:
schema_2 = Schema.new(columns=[
    Column.numeric("id"),
    Column.string("cat_names", is_nullable=False,has_commas=True,has_spaces=True),
    Column.string("cat_colours", is_nullable=False,has_commas=True,has_spaces=False)
])

fixer_2 = Fixer.new(schema_2)

parsed_example_2 = fixer_2.fix_file("../examples/example_2.csv", skip_first_line=False, show_possible_parses=True)

Multiple paths found at line index 0 - needs to be resolved.
Multiple paths found at line index 1 - needs to be resolved.


['1', 'chanom', 'chayen,orange,orange']
['1', 'chanom,chayen', 'orange,orange']
['1', 'chanom,chayen,orange', 'orange']
['2', 'chayen', 'olieang,orange,black']
['2', 'chayen,olieang', 'orange,black']
['2', 'chayen,olieang,orange', 'black']
File has been processed!
Number of total entries: 4            
 Number of invalid entries: 2


In [9]:
parsed_example_2.print_all_invalid_entries()

Index	Line entry
0	1,chanom,chayen,orange,orange
1	2,chayen,olieang,orange,black


We can see that for lines with multiple commas, the processing fails as it is unable to tell apart which tokens should be placed in which column. However, with `show_possible_parses` enabled, we can see the exact line and its possible parses.

In [10]:
parsed_example_2.export_to_csv_best_effort("../examples/example_2_parsed.csv")

In [11]:
! cat ../examples/example_2_parsed.csv

id,cat_names,cat_colours
3,muffin,orange
4,chanom,orange


Exporting the parsed dataset will only result in valid rows being exported.

## Example 3

In [12]:
schema_3 = Schema.new(columns=[
    Column.numeric("id"),
    Column.string("username",is_nullable=False,has_commas=False,has_spaces=False),
    Column.numeric("number_of_platforms"),
    Column.string("platforms",is_nullable=True,has_commas=True,has_spaces=False),
    Column.numeric("number_of_cats"),
    Column.string("cat_names",is_nullable=True,has_commas=True,has_spaces=True)
])

fixer_3 = Fixer.new(schema_3)

parsed_example_3 = fixer_3.fix_file("../examples/example_3.csv", skip_first_line=True, show_possible_parses=True)

File has been processed!
Number of total entries: 9            
 Number of invalid entries: 0


In [13]:
parsed_example_3.export_to_csv_best_effort("../examples/example_3_parsed.csv")

In [14]:
! cat ../examples/example_3_parsed.csv

id,username,number_of_platforms,platforms,number_of_cats,cat_names
1,john_appleseed,2,"facebook,instagram",1,Apple
2,john_wick,0,,0,
3,bob,1,instagram,2,"fluffy,fluffy sr."
4,jlaw,1,instagram,1,snowy
5,tay_fast,2,"instagram,youtube",3,"grey,benson,button"
6,tommyj,0,,1,mimi
7,jakeyh,2,"twitter,instagram",2,"mob,Psycho"
8,pujanf,0,,2,


Since there is a clear divider between the two comma columns, valid parsings can be produced and exported. 

## Example 4

If the values of a column with commas is known, i.e. the values came from a multiple choice question on a form, they can be specified to help identify whether a value can be placed within a column.

In [15]:
schema_4 = Schema.new(columns=[
    Column.string("favourite_cat_colours", is_nullable=False,has_commas=True,has_spaces=False, format=r"^(orange|black|tabby|white|calico)"),
    Column.string("favourite_colour_reason", is_nullable=False,has_commas=True,has_spaces=True, format=r"^(?!orange|black|tabby|white|calico)")
])

fixer_4 = Fixer.new(schema_4)

parsed_example_4 = fixer_4.fix_file("../examples/example_4.csv", skip_first_line=True, show_possible_parses=True)
parsed_example_4.print_all_invalid_entries()

File has been processed!
Number of total entries: 6            
 Number of invalid entries: 0
Index	Line entry


In [16]:
parsed_example_4.export_to_csv_best_effort(filepath="../examples/example_4_parsed.csv")

In [17]:
! cat ../examples/example_4_parsed.csv

favourite_cat_colours,cat_colours
"orange,calico","because orange cats are very silly,and calicos are very pretty"
black,because black cats are very sweet despite superstition
white,my cat is white so i like white cats (my cat)
"orange,tabby",I like tabby cats because they look like striped fish.
"orange,calico,black,white,tabby","I like all cat colours,why discriminate?"


By specifying the RegEx formatting of tokens that are expected in each column, it can help with parsing tokens into their respective columns. 

However, this can only be done for columns where their expected values are known. For text columns, this may not be as effective.

As seen in the example above, the last column is a text column and its contents can be random. In this case, we can try to differentiate from the previous column since we know the previous column's values and exclude all tokens which begin with items from the previous column.